# Milestone 1

### Project idea

We want to create an interactive visualization with a map of France and its regions. When hovering over/clicking on a region, one can for example see the wine types grown in that region, their average ratings and typical words used to describe them. 

### Ideas for visualisation

Just wanted to write down some thoughts I had on how this could be presented: 

As many of the current visualizations that exist is basically google maps with locations on them, it could be cool to create a clean map (just the borders and names of the provinces). Then add different icons that signify the different varieties (a symbol of some kind, maybe their coat of arms). Then you can click on a province, get that province enlarged and see the different wineries, some information about them (maybe). 

Would also be cool to show some statistics of some kind for that province and their wines potentially. 



### Dataset description

The data was scraped from WineEnthusiast on November 22nd, 2017. The dataset can be found here: https://www.kaggle.com/zynicide/wine-reviews.

| Column name | Description |
| --- | --- |
| country | The country that the wine is from. |
| description | A few sentences from a sommelier describing the wine's taste, smell, look, feel, etc. | 
| designation | The vineyard within the winery where the grapes that made the wine are from. | 
| points | The number of points WineEnthusiast rated the wine on a scale of 1-100 (though they say they only post reviews for wines that score >=80). | 
| price | The cost for a bottle of the wine. (Currency?) | 
| province | The province or state that the wine is from. |
| region_1 | The wine growing area in a province or state. | 
| region_2 | Sometimes there are more specific regions specified within a wine growing area. |
| taster_name | Name of the taster. |
| taster_twitter_handle | Twitter name of the taster. |
| title | The title of the wine. | 
| variety | The type of grapes used to make the wine. |
| winery | The winery that made the wine |


## Data Cleaning

In [1]:
# Import packages that are used
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
df = pd.read_csv('Data/winemag-data-130k-v2.csv', sep = ',')
france = df[df.country.str.contains('France', na=False)]
france.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
11,11,France,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer


In [3]:
france.isnull().sum()

Unnamed: 0                   0
country                      0
description                  0
designation               7563
points                       0
price                     4317
province                     0
region_1                    76
region_2                 22093
taster_name                265
taster_twitter_handle      265
title                        0
variety                      0
winery                       0
dtype: int64

We are not interested in the columns: 'Unnamed: 0' (it has no meaning), country (we know which country it is), region_2 (it has no values), taster_name & taster_twitter_handle (it's not important who wrote the review).

There are also a lot of missing values for designation and price, but for now we will keep these if they turn out to be of interest later on. Since there are 21099 rows in the dataset, it can be smart to stay clear of visualizing data about designation and price as about 30% of the data is missing. 

In [5]:
wines = france.drop(['Unnamed: 0', 'country', 'region_2', 'taster_name','taster_twitter_handle'], axis=1)\
    .rename(columns={'region_1':'region'}).reset_index(drop = True)
wines.head(3)

,description,designation,points,price,province,region,title,variety,winery
0,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
1,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
2,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer


### What should we focus on in the data? 

#### Variety 
Exploration of the different varities

In [41]:
varieties = wines.variety.value_counts()
varieties

Bordeaux-style Red Blend    4725
Chardonnay                  2808
Pinot Noir                  1966
Rosé                        1923
Champagne Blend             1243
                            ... 
Merlot-Grenache                1
Petit Meslier                  1
Ondenc                         1
Roussanne-Marsanne             1
Petit Verdot                   1
Name: variety, Length: 148, dtype: int64

There are 148 different varieties in total. Might be smart to focus on the ones that occur more in the dataset, and 148 is also a lot to visualize. Might be a good idea to have a treshhold of minimum 30 occurences. That will include 35 varieties.  

In [51]:
# Create dataset with only these varietes. 
condition = varieties > 30
wanted_varieties = varieties[condition]

wines_lim = wines.loc[wines.variety.isin(wanted_varieties.index)]

In [52]:
wines_lim.head(3)

,description,designation,points,price,province,region,title,variety,winery
0,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
1,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
2,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer


#### Winery
Exploration of the different wineries

In [53]:
wineries = wines.winery.value_counts()
wineries

Louis Latour                         199
Georges Duboeuf                      196
Albert Bichot                        117
Jean-Luc and Paul Aegerter           113
Chanson Père et Fils                 109
                                    ... 
Villa Ponciago                         1
Vincent Pinard                         1
Domaine La Consonnière                 1
Clos Marsalette                        1
Le Bord'eau du Clos des Moiselles      1
Name: winery, Length: 3864, dtype: int64

In [54]:
# We give the same treshhold as for varieties, that is 30
condition = wineries > 30
wanted_wineries = wineries[condition]

wines_lim = wines_lim.loc[wines_lim.winery.isin(wanted_wineries.index)].reset_index(drop = True)
wines_lim.head(3)

,description,designation,points,price,province,region,title,variety,winery
0,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
1,"This is a festive wine, with soft, ripe fruit ...",Nouveau,86,9.0,Beaujolais,Beaujolais,Henry Fessy 2012 Nouveau (Beaujolais),Gamay,Henry Fessy
2,"Soft and fruity, this is a generous, ripe wine...",Eté Indien,86,14.0,Beaujolais,Brouilly,Vignerons de Bel Air 2011 Eté Indien (Brouilly),Gamay,Vignerons de Bel Air


Have now reduced the dataset from 21099 to 6186 rows, which is also the amount after limiting the varieties. This means that no data is lost after removing the wineries with less than 30 occurences in the *france_lim* dataset. 

#### Price & Designation

In [55]:
wines_lim.price.value_counts(dropna = False)

NaN      714
20.0     221
25.0     180
15.0     174
30.0     158
        ... 
166.0      1
455.0      1
111.0      1
226.0      1
359.0      1
Name: price, Length: 255, dtype: int64

Now, 18% of the prices are missing. It should not be a problem to include the prices at this rate, but it may depend on the choice of visualization. 

In [56]:
wines_lim.designation.value_counts(dropna = False)

NaN                                                          975
Vieilles Vignes                                               93
Réserve                                                       66
Brut Rosé                                                     58
Collection                                                    54
                                                            ... 
Vorbourg Grand Cru Clos Saint Landelin Vendanges Tardives      1
Rasteau Vieilles Vignes                                        1
Les Vieilles Vignes de Sainte Claire                           1
Aramis Tannat Cabernet                                         1
Buissonnier Côte Chalonnaise Chardonnay                        1
Name: designation, Length: 1585, dtype: int64

About 22% of the designation is now missing. This is also something to consider when choosing a visualization. 

#### Province

In [57]:
wines_lim.province.value_counts()

Burgundy                1968
Alsace                  1189
Southwest France         434
Beaujolais               374
Loire Valley             312
Rhône Valley             249
Champagne                220
Provence                 118
France Other              96
Bordeaux                  79
Languedoc-Roussillon      72
Name: province, dtype: int64

All of the provinces are still represented in the dataset after cleaning of the original dataset. We have not lost any representation of areas. 

## Data Exploration

Group the dataset to get the average price and points for each type of variety in every region & province. Show also the amount of wines entailed in the calculation.

In [65]:
grouped = wines_lim.groupby(['province', 'region', 'variety'])\
    .agg({'price':'mean', 'points':'mean', 'title':'count'}).reset_index()
grouped

,province,region,variety,price,points,title
0,Alsace,Alsace,Alsace white blend,18.200000,88.590909,22
1,Alsace,Alsace,Chardonnay,22.666667,90.666667,3
2,Alsace,Alsace,Gewürztraminer,38.526087,90.271605,243
3,Alsace,Alsace,Muscat,37.045455,90.916667,24
4,Alsace,Alsace,Pinot Blanc,18.084507,87.431818,88
...,...,...,...,...,...,...
355,Southwest France,Vin de Pays du Comté Tolosan,Malbec,NaN,86.000000,2
356,Southwest France,Vin de Pays du Comté Tolosan,Red Blend,9.000000,86.000000,1
357,Southwest France,Vin de Pays du Comté Tolosan,Rosé,NaN,85.000000,1
358,Southwest France,Vin de Pays du Lot,Malbec,12.000000,84.000000,1


Group the dataset by province and region and retrieve all of the of varieties grown in each grouping.

In [66]:
provinces = wines_lim.groupby(['province', 'region']).agg({'variety':'unique'}).reset_index()
provinces

,province,region,variety
0,Alsace,Alsace,"[Pinot Gris, Riesling, Gewürztraminer, White B..."
1,Alsace,Crémant d'Alsace,"[Sparkling Blend, Pinot Noir, Chardonnay, Pino..."
2,Beaujolais,Beaujolais,[Gamay]
3,Beaujolais,Beaujolais Blanc,[Chardonnay]
4,Beaujolais,Beaujolais Rosé,"[Gamay, Rosé]"
...,...,...,...
214,Southwest France,Pacherenc du Vic Bilh,"[Gros and Petit Manseng, Petit Manseng]"
215,Southwest France,Saint-Mont,"[Red Blend, White Blend, Tannat]"
216,Southwest France,Vin de Pays des Côtes de Gascogne,"[White Blend, Sauvignon Blanc, Chardonnay, Tan..."
217,Southwest France,Vin de Pays du Comté Tolosan,"[Malbec, Rosé, Red Blend]"


Sort wines according to points and price.

In [70]:
sortedPointsPrice = wines_lim[['title', 'variety', 'province', 'region', 'points', 'price']].sort_values(by=['points', 'price'], ascending=(False, True))
sortedPointsPrice

,title,variety,province,region,points,price
3469,Louis Roederer 2008 Cristal Vintage Brut (Cha...,Champagne Blend,Champagne,Champagne,100,250.0
1943,Louis Jadot 2005 Charmes-Chambertin,Pinot Noir,Burgundy,Charmes-Chambertin,98,134.0
1944,Louis Jadot 2005 Clos de la Roche,Pinot Noir,Burgundy,Clos de la Roche,98,138.0
3907,M. Chapoutier 1999 Le Méal Ermitage (Hermitage),Rhône-style Red Blend,Rhône Valley,Hermitage,98,150.0
1074,Louis Jadot 2014 Bâtard-Montrachet,Chardonnay,Burgundy,Bâtard-Montrachet,98,367.0
...,...,...,...,...,...,...
889,Bougrier 2013 V Douce France (Vouvray),Chenin Blanc,Loire Valley,Vouvray,81,14.0
725,Georges Duboeuf 2001 Vin de Pays de l'Ardeche ...,Viognier,France Other,France,80,10.0
1391,Bougrier 2015 Gamay-Grolleau (Rosé d'Anjou),Rosé,Loire Valley,Rosé d'Anjou,80,11.0
4463,Cave de Ribeauvillé NV Giersberger Brut Sparkl...,Sparkling Blend,Alsace,Crémant d'Alsace,80,14.0


Calculate best wine quality for money.

In [71]:
bestValue = wines_lim[['title', 'variety', 'province', 'region', 'points', 'price']]
bestValue['points/price'] = wines_lim['points'] / wines_lim['price']
bestValue = bestValue.sort_values(by=['points/price'], ascending=False)
bestValue

/Users/verafristedt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,variety,province,region,points,price,points/price
4596,Producteurs Plaimont 2013 Terres d'Artagnan Cu...,White Blend,Southwest France,Côtes de Gascogne,86,7.0,12.285714
2309,Barton & Guestier 2013 Réserve Sauvignon Blanc...,Sauvignon Blanc,Southwest France,Côtes de Gascogne,85,7.0,12.142857
4359,Barton & Guestier 2010 Reserve Sauvignon Blanc...,Sauvignon Blanc,Southwest France,Côtes de Gascogne,86,8.0,10.750000
2307,Producteurs Plaimont 2013 Colombelle L'Origine...,Red Blend,Southwest France,Côtes de Gascogne,85,8.0,10.625000
836,Producteurs Plaimont 2007 Colombelle White Whi...,White Blend,Southwest France,Vin de Pays des Côtes de Gascogne,85,8.0,10.625000
...,...,...,...,...,...,...,...
5078,Lionel Osmin & Cie 2011 Pont du Diable Malbec ...,Malbec,France Other,Vin de France,89,NaN,NaN
5081,Château Tour des Gendres 2011 Moulin des Dames...,Bordeaux-style White Blend,Southwest France,Bergerac Sec,90,NaN,NaN
5088,Henri de Villamont 2014 Les Baudes Premier Cru...,Pinot Noir,Burgundy,Chambolle-Musigny,93,NaN,NaN
5090,René Muré 2011 Côte de Rouffach Pinot Noir (Al...,Pinot Noir,Alsace,Alsace,87,NaN,NaN
